In [1]:
# Copyright 2019 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

<img src="http://developer.download.nvidia.com/compute/machine-learning/frameworks/nvidia_logo.png" style="width: 90px; float: right;">

# TRTorch Getting Started - ResNet 50

## Overview

In the practice of developing machine learning models, there are few tools as approachable as PyTorch for developing and experimenting in designing machine learning models. The power of PyTorch comes from its deep integration into Python, its flexibility and its approach to automatic differentiation and execution (eager execution). However, when moving from research into production, the requirements change and we may no longer want that deep Python integration and we want optimization to get the best performance we can on our deployment platform. In PyTorch 1.0, TorchScript was introduced as a method to separate your PyTorch model from Python, make it portable and optimizable. TorchScript uses PyTorch's JIT compiler to transform your normal PyTorch code which gets interpreted by the Python interpreter to an intermediate representation (IR) which can have optimizations run on it and at runtime can get interpreted by the PyTorch JIT interpreter. For PyTorch this has opened up a whole new world of possibilities, including deployment in other languages like C++. It also introduces a structured graph based format that we can use to do down to the kernel level optimization of models for inference.

When deploying on NVIDIA GPUs TensorRT, NVIDIA's Deep Learning Optimization SDK and Runtime is able to take models from any major framework and specifically tune them to perform better on specific target hardware in the NVIDIA family be it an A100, TITAN V, Jetson Xavier or NVIDIA's Deep Learning Accelerator. TensorRT performs a couple sets of optimizations to achieve this. TensorRT fuses layers and tensors in the model graph, it then uses a large kernel library to select implementations that perform best on the target GPU. TensorRT also has strong support for reduced operating precision execution which allows users to leverage the Tensor Cores on Volta and newer GPUs as well as reducing memory and computation footprints on device.

TRTorch is a compiler that uses TensorRT to optimize TorchScript code, compiling standard TorchScript modules into ones that internally run with TensorRT optimizations. This enables you to continue to remain in the PyTorch ecosystem, using all the great features PyTorch has such as module composability, its flexible tensor implementation, data loaders and more. TRTorch is available to use with both PyTorch and LibTorch.

### Learning objectives

This notebook demonstrates the steps for compiling a TorchScript module with TRTorch on a pretrained ResNet-50 network, and running it to test the speedup obtained.

## Content
1. [Requirements](#1)
1. [ResNet-50 Overview](#2)
1. [Creating TorchScript modules](#3)
1. [Compiling with TRTorch](#4)
1. [Running Inference](#5)
1. [Measuring Speedup](#6)
1. [Conclusion](#7)

<a id="1"></a>
## 1. Requirements

Follow the steps in `notebooks/README` to prepare a Docker container, within which you can run this notebook.

<a id="2"></a>
## 2. ResNet-50 Overview


PyTorch has a model repository called the PyTorch Hub, which is a source for high quality implementations of common models. We can get our ResNet-50 model from there pretrained on ImageNet.

### Model Description

This ResNet-50 model is based on the [Deep Residual Learning for Image Recognition](https://arxiv.org/pdf/1512.03385.pdf) paper, which describes ResNet as “a method for detecting objects in images using a single deep neural network". The input size is fixed to 32x32.

(More information about the ResNet-50 model is available at Nvidia's "[DeepLearningExamples](https://github.com/NVIDIA/DeepLearningExamples/tree/master/PyTorch/Classification/ConvNets/resnet50v1.5)" Github.)




In [2]:
import torch

torch.hub.list('NVIDIA/DeepLearningExamples')

Downloading: "https://github.com/NVIDIA/DeepLearningExamples/archive/master.zip" to /root/.cache/torch/hub/master.zip


['nvidia_ncf',
 'nvidia_ssd',
 'nvidia_ssd_processing_utils',
 'nvidia_tacotron2',
 'nvidia_waveglow',
 'relocated']

In [34]:
import torch

resnet50_model = torch.hub.load('pytorch/vision:v0.6.0', 'resnet50', pretrained=True)
# resnet50_model = torchvision.models.resnet50(pretrained=False, progress=True)

# model = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_ssd', model_math="fp32")
# model = model.eval().to("cuda")

Downloading: "https://github.com/pytorch/vision/archive/v0.6.0.zip" to /root/.cache/torch/hub/v0.6.0.zip


### Benchmark utility

Let us define a helper function to benchmark a model.

In [35]:
import time
import numpy as np

import torch.backends.cudnn as cudnn
cudnn.benchmark = True

def benchmark(model, input_shape=(1024, 1, 32, 32), dtype='fp32', nwarmup=50, nruns=10000):
    input_data = torch.randn(input_shape)
    input_data = input_data.to("cuda")
    if dtype=='fp16':
        input_data = input_data.half()
        
    print("Warm up ...")
    with torch.no_grad():
        for _ in range(nwarmup):
            features = model(input_data)
    torch.cuda.synchronize()
    print("Start timing ...")
    timings = []
    with torch.no_grad():
        for i in range(1, nruns+1):
            start_time = time.time()
            features = model(input_data)
            torch.cuda.synchronize()
            end_time = time.time()
            timings.append(end_time - start_time)
            if i%100==0:
                print('Iteration %d/%d, ave batch time %.2f ms'%(i, nruns, np.mean(timings)*1000))

    print("Input shape:", input_data.size())
    print("Output features size:", features.size())
    print('Average batch time: %.2f ms'%(np.mean(timings)*1000))

In [36]:
# Model benchmark without TRTorch/TensorRT
model = resnet50_model.eval().to("cuda")
benchmark(model, input_shape=(128, 3, 32, 32), nruns=1000)

Warm up ...
Start timing ...
Iteration 100/1000, ave batch time 14.86 ms
Iteration 200/1000, ave batch time 14.80 ms
Iteration 300/1000, ave batch time 14.77 ms
Iteration 400/1000, ave batch time 14.75 ms
Iteration 500/1000, ave batch time 14.74 ms
Iteration 600/1000, ave batch time 14.71 ms
Iteration 700/1000, ave batch time 14.71 ms
Iteration 800/1000, ave batch time 14.55 ms
Iteration 900/1000, ave batch time 14.40 ms
Iteration 1000/1000, ave batch time 14.28 ms
Input shape: torch.Size([128, 3, 32, 32])
Output features size: torch.Size([128, 1000])
Average batch time: 14.28 ms


<a id="3"></a>
## 3. Creating TorchScript modules

To compile with TRTorch, the model must first be in **TorchScript**. TorchScript is a programming language included in PyTorch which removes the Python dependency normal PyTorch models have. This conversion is done via a JIT compiler which given a PyTorch Module will generate an equivalent TorchScript Module. There are two paths that can be used to generate TorchScript: **Tracing** and **Scripting**. Tracing follows execution of PyTorch generating ops in TorchScript corresponding to what it sees. Scripting does an analysis of the Python code and generates TorchScript, this allows the resulting graph to include control flow which tracing cannot do. Tracing however due to its simplicity is more likely to compile successfully with TRTorch (though both systems are supported). 

In [37]:
model = resnet50_model.eval().to("cuda")
traced_model = torch.jit.trace(model, [torch.randn((128, 3, 32, 32)).to("cuda")])

We can save this model and use it independently of Python.

In [38]:
# This is just an example, and not required for the purposes of this demo
torch.jit.save(traced_model, "resnet_50_traced.jit.pt")

In [39]:
# Obtain the average time taken by a batch of input
benchmark(traced_model, input_shape=(128, 3, 32, 32), nruns=1000)

Warm up ...
Start timing ...
Iteration 100/1000, ave batch time 11.16 ms
Iteration 200/1000, ave batch time 11.17 ms
Iteration 300/1000, ave batch time 11.17 ms
Iteration 400/1000, ave batch time 11.18 ms
Iteration 500/1000, ave batch time 11.18 ms
Iteration 600/1000, ave batch time 11.18 ms
Iteration 700/1000, ave batch time 11.18 ms
Iteration 800/1000, ave batch time 11.18 ms
Iteration 900/1000, ave batch time 11.18 ms
Iteration 1000/1000, ave batch time 11.18 ms
Input shape: torch.Size([128, 3, 32, 32])
Output features size: torch.Size([128, 1000])
Average batch time: 11.18 ms


<a id="4"></a>
## 4. Compiling with TRTorch

TorchScript modules behave just like normal PyTorch modules and are intercompatable. From TorchScript we can now compile a TensorRT based module. This module will still be implemented in TorchScript but all the computation will be done in TensorRT.

In [40]:
import trtorch

# The compiled module will have precision as specified by "op_precision".
# Here, it will have FP16 precision.
trt_model = trtorch.compile(traced_model, {
    "input_shapes": [(1, 3, 32, 32)],
    "op_precision": torch.half, # Run with FP16
    "workspace_size": 1 << 20
})


<a id="5"></a>
## 5. Running Inference

We can now run inference on the TRTorch compiled model. This is demonstrated below using sample images from the COCO 2017 Validation set.

In [41]:
# Download an example image from the pytorch website
import urllib
url, filename = ("https://github.com/pytorch/hub/raw/master/dog.jpg", "dog.jpg")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)

In [55]:
# sample execution (requires torchvision)
from PIL import Image
from torchvision import transforms
input_image = Image.open(filename)
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

# move the input and model to GPU for speed if available
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

with torch.no_grad():
    output = model(input_batch)
# Tensor of shape 1000, with confidence scores over Imagenet's 1000 classes
print(output[0])
# The output has unnormalized scores. To get probabilities, you can run a softmax on it.
print(torch.nn.functional.softmax(output[0], dim=0))

AttributeError: module 'torch.jit' has no attribute '_script_if_tracing'

In [54]:
# !pip uninstall torchvision
# !/usr/bin/python -m pip install --upgrade pip

!pip install torch===1.6.0 torchvision===0.4.1 -f https://download.pytorch.org/whl/torch_stable.html
# !pip install torchvision==0.5.0 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 10.1 MB 5.3 MB/s eta 0:00:01
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.5.0
    Uninstalling torchvision-0.5.0:
      Successfully uninstalled torchvision-0.5.0
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

torchvision 0.4.1 requires torch==1.3.0, but you'll have torch 1.6.0 which is incompatible.


In [30]:
# Sample images from the COCO validation set
uris = [
    'http://images.cocodataset.org/val2017/000000397133.jpg',
    'http://images.cocodataset.org/val2017/000000037777.jpg',
    'http://images.cocodataset.org/val2017/000000252219.jpg'
]

# For convenient and comprehensive formatting of input and output of the model, load a set of utility methods.
utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_ssd_processing_utils')

# Format images to comply with the network input
inputs = [utils.prepare_input(uri) for uri in uris]
tensor = utils.prepare_tensor(inputs, False)

# The model was trained on COCO dataset, which we need to access in order to
# translate class IDs into object names. 
classes_to_labels = utils.get_coco_object_dictionary()

Downloading: "https://github.com/NVIDIA/DeepLearningExamples/archive/torchhub.zip" to /root/.cache/torch/hub/torchhub.zip


ModuleNotFoundError: No module named 'skimage'

In [11]:
benchmark(model)

Warm up ...
Start timing ...
Iteration 1000/10000, ave batch time 0.93 ms
Iteration 2000/10000, ave batch time 0.93 ms
Iteration 3000/10000, ave batch time 0.93 ms
Iteration 4000/10000, ave batch time 0.93 ms
Iteration 5000/10000, ave batch time 0.93 ms
Iteration 6000/10000, ave batch time 0.93 ms
Iteration 7000/10000, ave batch time 0.93 ms
Iteration 8000/10000, ave batch time 0.93 ms
Iteration 9000/10000, ave batch time 0.93 ms
Iteration 10000/10000, ave batch time 0.93 ms
Input shape: torch.Size([1024, 1, 32, 32])
Output features size: torch.Size([1024, 10])
Average batch time: 0.93 ms


When compiling your module to TorchScript, there are two paths: Tracing and Scripting.  
 
### Tracing

Tracing follows the path of execution when the module is called and records what happens. This recording is what the TorchScript IR will describe. To trace an instance of our LeNet module, we can call torch.jit.trace  with an example input. 

In [12]:
traced_model = torch.jit.trace(model, torch.empty([1,1,32,32]).to("cuda"))
traced_model

LeNet(
  original_name=LeNet
  (feat): LeNetFeatExtractor(
    original_name=LeNetFeatExtractor
    (conv1): Conv2d(original_name=Conv2d)
    (conv2): Conv2d(original_name=Conv2d)
  )
  (classifer): LeNetClassifier(
    original_name=LeNetClassifier
    (fc1): Linear(original_name=Linear)
    (fc2): Linear(original_name=Linear)
    (fc3): Linear(original_name=Linear)
  )
)

In [13]:
benchmark(traced_model)

Warm up ...
Start timing ...
Iteration 1000/10000, ave batch time 0.68 ms
Iteration 2000/10000, ave batch time 0.68 ms
Iteration 3000/10000, ave batch time 0.68 ms
Iteration 4000/10000, ave batch time 0.68 ms
Iteration 5000/10000, ave batch time 0.68 ms
Iteration 6000/10000, ave batch time 0.68 ms
Iteration 7000/10000, ave batch time 0.68 ms
Iteration 8000/10000, ave batch time 0.68 ms
Iteration 9000/10000, ave batch time 0.68 ms
Iteration 10000/10000, ave batch time 0.68 ms
Input shape: torch.Size([1024, 1, 32, 32])
Output features size: torch.Size([1024, 10])
Average batch time: 0.68 ms


### Scripting

Scripting actually inspects your code with a compiler and  generates an equivalent TorchScript program. The difference is that since tracing simply follows the execution of your module, it cannot pick up control flow for instance, it will only follow the code path that a particular input triggers. By working from the Python code, the compiler can include these components. We can run the script compiler on our LeNet  module by calling torch.jit.script.


In [14]:
model = LeNet().to("cuda").eval()
script_model = torch.jit.script(model)


In [15]:
script_model

RecursiveScriptModule(
  original_name=LeNet
  (feat): RecursiveScriptModule(
    original_name=LeNetFeatExtractor
    (conv1): RecursiveScriptModule(original_name=Conv2d)
    (conv2): RecursiveScriptModule(original_name=Conv2d)
  )
  (classifer): RecursiveScriptModule(
    original_name=LeNetClassifier
    (fc1): RecursiveScriptModule(original_name=Linear)
    (fc2): RecursiveScriptModule(original_name=Linear)
    (fc3): RecursiveScriptModule(original_name=Linear)
  )
)

In [16]:
benchmark(script_model)

Warm up ...
Start timing ...
Iteration 1000/10000, ave batch time 0.68 ms
Iteration 2000/10000, ave batch time 0.68 ms
Iteration 3000/10000, ave batch time 0.68 ms
Iteration 4000/10000, ave batch time 0.68 ms
Iteration 5000/10000, ave batch time 0.68 ms
Iteration 6000/10000, ave batch time 0.68 ms
Iteration 7000/10000, ave batch time 0.68 ms
Iteration 8000/10000, ave batch time 0.68 ms
Iteration 9000/10000, ave batch time 0.68 ms
Iteration 10000/10000, ave batch time 0.68 ms
Input shape: torch.Size([1024, 1, 32, 32])
Output features size: torch.Size([1024, 10])
Average batch time: 0.68 ms


<a id="3"></a>
## 3. Compiling with TRTorch

### TorchScript traced model

First, we compile the TorchScript traced model with TRTorch. Notice the performance impact.

In [17]:
import trtorch

compile_settings = {
    "input_shapes": [
        {
            "min" : [1, 1, 32, 32],
            "opt" : [1, 1, 33, 33],
            "max" : [1, 1, 34, 34],
        }
    ],
    "op_precision": torch.half # Run with FP16
}

trt_ts_module = trtorch.compile(traced_model, compile_settings)

input_data = torch.randn((1, 1, 32, 32))
input_data = input_data.half().to("cuda")

input_data = input_data.half()
result = trt_ts_module(input_data)
torch.jit.save(trt_ts_module, "trt_ts_module.ts")

In [18]:
benchmark(trt_ts_module, dtype="fp16")

Warm up ...
Start timing ...
Iteration 1000/10000, ave batch time 0.24 ms
Iteration 2000/10000, ave batch time 0.24 ms
Iteration 3000/10000, ave batch time 0.24 ms
Iteration 4000/10000, ave batch time 0.24 ms
Iteration 5000/10000, ave batch time 0.24 ms
Iteration 6000/10000, ave batch time 0.24 ms
Iteration 7000/10000, ave batch time 0.23 ms
Iteration 8000/10000, ave batch time 0.23 ms
Iteration 9000/10000, ave batch time 0.23 ms
Iteration 10000/10000, ave batch time 0.23 ms
Input shape: torch.Size([1024, 1, 32, 32])
Output features size: torch.Size([1, 10])
Average batch time: 0.23 ms


### TorchScript script model

Next, we compile the TorchScript script model with TRTorch. Notice the performance impact.

In [19]:
import trtorch

compile_settings = {
    "input_shapes": [
        {
            "min" : [1, 1, 32, 32],
            "opt" : [1, 1, 33, 33],
            "max" : [1, 1, 34, 34],
        }
    ],
    "op_precision": torch.half # Run with FP16
}

trt_script_module = trtorch.compile(script_model, compile_settings)

input_data = torch.randn((1, 1, 32, 32))
input_data = input_data.half().to("cuda")

input_data = input_data.half()
result = trt_script_module(input_data)
torch.jit.save(trt_script_module, "trt_script_module.ts")

In [20]:
benchmark(trt_ts_module, dtype="fp16")

Warm up ...
Start timing ...
Iteration 1000/10000, ave batch time 0.20 ms
Iteration 2000/10000, ave batch time 0.20 ms
Iteration 3000/10000, ave batch time 0.20 ms
Iteration 4000/10000, ave batch time 0.21 ms
Iteration 5000/10000, ave batch time 0.21 ms
Iteration 6000/10000, ave batch time 0.21 ms
Iteration 7000/10000, ave batch time 0.21 ms
Iteration 8000/10000, ave batch time 0.21 ms
Iteration 9000/10000, ave batch time 0.21 ms
Iteration 10000/10000, ave batch time 0.21 ms
Input shape: torch.Size([1024, 1, 32, 32])
Output features size: torch.Size([1, 10])
Average batch time: 0.21 ms


## Conclusion

In this notebook, we have walked through the complete process of compiling TorchScript models with TRTorch and test the performance impact of the optimization.

### What's next
Now it's time to try TRTorch on your own model. Fill out issues at https://github.com/NVIDIA/TRTorch. Your involvement will help future development of TRTorch.
